### Measuring time

The official docs are given [here](https://docs.python.org/3/library/time.html)

#### time.perf_counter() → float

Return the value (in fractional seconds) of a performance counter, i.e. a clock with the highest available resolution to measure a short duration. It does include time elapsed during sleep and is system-wide. The reference point of the returned value is undefined, so that only the difference between the results of two calls is valid.

In [12]:
"""Measuring time"""

from time import perf_counter


def upto_for(n):
    """Sum 1...n with a for loop"""
    total = 0
    for i in range(n):
        total += i
    return total


def upto_sum(n):
    """Sum 1...n with built-in sum and range"""
    return sum(range(n))


if __name__ == '__main__':
    n = 1_000_000                 # haha python version 3.6+

    start = perf_counter()
    upto_for(n)
    duration = perf_counter() - start
    print('upto_for', duration)

    start = perf_counter()
    upto_sum(n)
    duration = perf_counter() - start
    print('upto_sum', duration)


upto_for 0.11639189200013789
upto_sum 0.03240389700022206


### timeit

This module provides a simple way to time small bits of Python code. It has both a Command-Line Interface as well as a callable one. It avoids a number of common traps for measuring execution times. 

In [13]:
"""Using "timeit"""
from timeit import timeit

items = {
    'a': 1,
    'b': 2,
}
default = -1


def use_catch(key):
    """Use try/catch to get a key with default"""
    try:
        return items[key]
    except KeyError:
        return default


def use_get(key):
    """Use dict.get to get a key with default"""
    return items.get(key, default)


if __name__ == '__main__':
    # Key is in the dictionary
    print('catch', timeit('use_catch("a")', 'from __main__ import use_catch'))
    print('get', timeit('use_get("a")', 'from __main__ import use_get'))

    # Key is missing from the dictionary
    print('catch', timeit('use_catch("x")', 'from __main__ import use_catch'))
    print('get', timeit('use_get("x")', 'from __main__ import use_get'))


catch 0.21018564899986814
get 0.290800427999784
catch 0.6554438749999463
get 0.30579440600013186


Inference : If the key is in the dictionary try/catch method is faster. If the key is not in the dictionary then, get method is faster.

main*›$ ipython

Python 3.7.10 (default, Feb 26 2021, 18:47:35) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.18.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: run -n using_timeit.py

In [2]: %timeit use_get('a')

317 ns ± 1.26 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)



In [3]: %timeit use_catch('a')

209 ns ± 0.679 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Profiler:

A profiler moniters the execution of the code and records the where it spends its time.

## CPU profiling 


* cProfile is recommended by python
* Deterministic profilers record every function call, return, and exception
* Statistical profilers record where the program is at small intervals
* pstats module displays profiler-generated statistics file

worth reading docs: [here](https://docs.python.org/3/library/profile.html)


In [15]:
### login.py

"""Example login code"""
from crypt import crypt
import sqlite3

salt = '$6$ZmBkxkRFj03LQOvr'  # Bad security, store safely
db = sqlite3.connect('passwords.db')
db.row_factory = sqlite3.Row  # Access columns by names


def user_passwd(user):
    """Get user password from db"""
    cur = db.cursor()
    cur.execute('SELECT passwd FROM users WHERE user = ?', (user, ))
    row = cur.fetchone()
    if row is None:  # No such user
        raise KeyError(user)
    return row['passwd']


def encrypt_passwd(passwd):
    """Encrypt user password"""
    return crypt(passwd, salt)


def login(user, password):
    """Return True is user/password pair matches"""
    try:
        db_passwd = user_passwd(user)
    except KeyError:
        return False

    passwd = encrypt_passwd(password)
    return passwd == db_passwd


In [ ]:
### prof.py

"""Example using cProfile"""
from login import login
from random import random


def gen_cases(n):
    """Generate tests cases"""
    for i in range(n):
        if random() > 0.1:  # 90% of logins are OK
            yield ('daffy', 'rabbit season')
        else:
            if random() < 0.2:
                yield ('tweety', 'puddy tat')  # no such user
            else:
                yield ('daffy', 'duck season')


def bench_login(cases):
    """Benchmark login with test cases"""
    for user, passwd in cases:
        login(user, passwd)


if __name__ == '__main__':
    n = 1000
    cases = list(gen_cases(n))

    if 1:
        bench_login(cases)

    if 0:
        import cProfile
        cProfile.run('bench_login(cases)')

    if 0:
        import cProfile
        cProfile.run('bench_login(cases)', filename = 'prof.out')

#### Illustrative in terminal
run this command.

python -m cProfile prof.py 

18130 function calls (17961 primitive calls) in 5.255 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   
   _22      0.000_  _0.000_    _0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)

...

In [ ]:
### prof.py

"""Example using cProfile"""
from login import login
from random import random


def gen_cases(n):
    """Generate tests cases"""
    for i in range(n):
        if random() > 0.1:  # 90% of logins are OK
            yield ('daffy', 'rabbit season')
        else:
            if random() < 0.2:
                yield ('tweety', 'puddy tat')  # no such user
            else:
                yield ('daffy', 'duck season')


def bench_login(cases):
    """Benchmark login with test cases"""
    for user, passwd in cases:
        login(user, passwd)


if __name__ == '__main__':
    n = 1000
    cases = list(gen_cases(n))

    if 0:
        bench_login(cases)

    if 1:
        import cProfile
        cProfile.run('bench_login(cases)')

    if 0:
        import cProfile
        cProfile.run('bench_login(cases)', filename = 'prof.out')

In [ ]:
#### now running cProfiling only to bench_login cases

python prof.py 

we get,
        8932 function calls in 5.155 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.155    5.155 <string>:1(<module>)
      982    0.002    0.000    5.104    0.005 crypt.py:60(crypt)
     1000    0.003    0.000    0.044    0.000 login.py:10(user_passwd)
      982    0.001    0.000    5.105    0.005 login.py:20(encrypt_passwd)
     1000    0.004    0.000    5.153    0.005 login.py:25(login)
        1    0.002    0.002    5.155    5.155 prof.py:18(bench_login)
      982    5.101    0.005    5.101    0.005 {built-in method _crypt.crypt}
        1    0.000    0.000    5.155    5.155 {built-in method builtins.exec}
      982    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
     1000    0.003    0.000    0.003    0.000 {method 'cursor' of 'sqlite3.Connection' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
     1000    0.031    0.000    0.031    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
     1000    0.006    0.000    0.006    0.000 {method 'fetchone' of 'sqlite3.Cursor' objects}



In [ ]:
### prof.py

"""Example using cProfile"""
from login import login
from random import random


def gen_cases(n):
    """Generate tests cases"""
    for i in range(n):
        if random() > 0.1:  # 90% of logins are OK
            yield ('daffy', 'rabbit season')
        else:
            if random() < 0.2:
                yield ('tweety', 'puddy tat')  # no such user
            else:
                yield ('daffy', 'duck season')


def bench_login(cases):
    """Benchmark login with test cases"""
    for user, passwd in cases:
        login(user, passwd)


if __name__ == '__main__':
    n = 1000
    cases = list(gen_cases(n))

    if 0:
        bench_login(cases)

    if 0:
        import cProfile
        cProfile.run('bench_login(cases)')

    if 1:
        import cProfile
        cProfile.run('bench_login(cases)', filename = 'prof.out')

In [ ]:
#### now running cProfiling only to bench_login cases and saving results to output file

python prof.py --> no output, but saved n file
python -m pstats prof.out

we get into pstats browser,

Welcome to the profile statistics browser.
prof.out% stats 10
Tue May 11 11:01:24 2021    prof.out

         8932 function calls in 5.118 seconds

   Random listing order was used
   List reduced from 13 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.118    5.118 {built-in method builtins.exec}
      982    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
     1000    0.027    0.000    0.027    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
     1000    0.005    0.000    0.005    0.000 {method 'fetchone' of 'sqlite3.Cursor' objects}
     1000    0.002    0.000    0.002    0.000 {method 'cursor' of 'sqlite3.Connection' objects}
        1    0.000    0.000    5.118    5.118 <string>:1(<module>)
      982    0.002    0.000    5.074    0.005 /home/simple/anaconda3/envs/StriveSchool/lib/python3.7/crypt.py:60(crypt)
     1000    0.003    0.000    0.038    0.000 /home/simple/ai/AI/basics/optimizing_code_python/chapter1/01_03/login.py:10(user_passwd)
     1000    0.003    0.000    5.116    0.005 /home/simple/ai/AI/basics/optimizing_code_python/chapter1/01_03/login.py:25(login)

sort cumtime
stats 10


Tue May 11 11:01:24 2021    prof.out

         8932 function calls in 5.118 seconds

   Ordered by: cumulative time
   List reduced from 13 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.118    5.118 {built-in method builtins.exec}
        1    0.000    0.000    5.118    5.118 <string>:1(<module>)
        1    0.001    0.001    5.118    5.118 prof.py:18(bench_login)
     1000    0.003    0.000    5.116    0.005 /home/simple/ai/AI/basics/optimizing_code_python/chapter1/01_03/login.py:25(login)
      982    0.001    0.000    5.075    0.005 /home/simple/ai/AI/basics/optimizing_code_python/chapter1/01_03/login.py:20(encrypt_passwd)
      982    0.002    0.000    5.074    0.005 /home/simple/anaconda3/envs/StriveSchool/lib/python3.7/crypt.py:60(crypt)
      982    5.072    0.005    5.072    0.005 {built-in method _crypt.crypt}
     1000    0.003    0.000    0.038    0.000 /home/simple/ai/AI/basics/optimizing_code_python/chapter1/01_03/login.py:10(user_passwd)
     1000    0.027    0.000    0.027    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
     1000    0.005    0.000    0.005    0.000 {method 'fetchone' of 'sqlite3.Cursor' objects}




In [16]:
#https://docs.python.org/3/library/profile.html

try visualization --> with snakeviz

snakeviz prof.out


https://jiffyclub.github.io/snakeviz/#snakeviz

worth looking light weight profilers:

https://pythonspeed.com/articles/beyond-cprofile/
